# Лабораторная работа 3. 
## Логинов Сергей
### Вариант 6

1. Подключимся к БД.

In [1]:
import sqlite3

db_connection = sqlite3.connect('instacart.db')
db_cursor = db_connection.cursor()

2. При помощи запроса SELECT извлечем из таблицы order_products__train записи, соответствующие указанным в индивидуальном задании дню недели (1) и коду департамента (18). Определим количество записей в полученном наборе и определим количество товаров (поле order_id таблицы order_products__train) в транзакциях набора.

In [2]:
query1 = "SELECT count(DISTINCT train.order_id) \
    FROM order_products__train as train \
        join orders as o on train.order_id == o.order_id \
        join products p on train.product_id = p.product_id \
        join departments d on p.department_id = d.department_id \
    where p.department_id = 18 and o.order_dow = 1"
query2 = "SELECT count(*) \
    FROM order_products__train as train \
        join orders as o on train.order_id == o.order_id \
        join products p on train.product_id = p.product_id \
        join departments d on p.department_id = d.department_id \
    where p.department_id = 18 and o.order_dow = 1"
count1 = db_cursor.execute(query1).fetchone()[0]
count2 = db_cursor.execute(query2).fetchone()[0]
print("Количество записей: {}\nКоличество товаров: {}".format(count2, count1))

Количество записей: 2186
Количество товаров: 944


3. Определим количество покупок для пяти наиболее популярных товаров за период, который дан в задании (1 день недели), с кодом департамента 18.

In [3]:
query3 = "SELECT count(train.order_id) counter, p.product_name, p.product_id \
        FROM order_products__train as train \
        join orders as o on train.order_id == o.order_id \
        join products p on train.product_id = p.product_id \
        join departments d on p.department_id = d.department_id \
    where p.department_id = 18 and o.order_dow = 1 \
    GROUP BY p.product_id \
    ORDER BY count(train.order_id) DESC limit 5"
db_cursor.execute(query3)
print("5 наиболее популярных товаров:")
while True:
    r = db_cursor.fetchone()
    if r:
        print("Товар = {}(id={}); количество покупок = {}".format(r[1], r[2], r[0]))
    else:
        break

5 наиболее популярных товаров:
Товар = Baby Food Stage 2 Blueberry Pear & Purple Carrot(id=43875); количество покупок = 45
Товар = Organic Mixed Berry Yogurt & Fruit Snack(id=21162); количество покупок = 38
Товар = Kids Sensible Foods Broccoli Littles(id=49215); количество покупок = 37
Товар = Gluten Free SpongeBob Spinach Littles(id=2611); количество покупок = 34
Товар = Broccoli & Apple Stage 2 Baby Food(id=3020); количество покупок = 32


4. Построим транзакционную базу данных для поиск ассоциативных правил из полученного набора записей таблицы order_products__train, используя в качестве идентификатора транзакции поле order_id, а в качестве названий товаров - поле product_name из таблицы products, соответствующее полю product_id. Заменим пробелы в текстовых переменных на +.

In [4]:
query4 = "SELECT o.order_id, p.product_name \
FROM order_products__train as train \
    join orders as o on train.order_id == o.order_id \
    join products p on train.product_id = p.product_id \
    join departments d on p.department_id = d.department_id \
where p.department_id = 18 and o.order_dow = 1 \
ORDER BY  o.order_id"
db_cursor.execute(query4)

tdbase = []
I = []
j = 0
data = db_cursor.fetchone()
tdbase.append([data[0], {data[1]}])
while True:
    data = db_cursor.fetchone()
    if data:
        if data[1] not in I:
            I.append(data[1].replace(" ", "+"))
        if data[0] == tdbase[j][0]:
            tdbase[j][1].add(data[1].replace(" ", "+"))
        else:
            new_data = [data[0], {data[1].replace(" ", "+")}]
            tdbase.append(new_data)
            j += 1
    else:
        break
I.sort()

In [5]:
tdbase

[['1008360',
  {'1st Foods Peaches',
   '1st+Foods+Baby+Food-+Pears',
   'Baby+Dry+Diapers+Jumbo+Pack+Size+4',
   'Infant+Formula+Powder',
   'Organic+Baby+Brown+Rice+Cereal'}],
 ['1015670',
  {'Baby+Food+Stage+2+Pumpkin+Banana',
   'Mighty+4+Kale,+Strawberry,+Amaranth+&+Greek+Yogurt+Tots+Snack',
   'Mighty+4+Sweet+Potato,+Blueberry,+Millet+&+Greek+Yogurt+Tots+Snack',
   'Stage+2+Cherry,+Sweet+Corn,+and+Greek+Yogurt+Baby+Food'}],
 ['1018282',
  {'Baby+Food+Stage+2+Blueberry+Pear+&+Purple+Carrot',
   'Sweet+Potato+&+Chicken+Dinner'}],
 ['1021926', {'Vanilla+Bottles', 'White+Buttermints'}],
 ['1022993', {'Gobble+Gobble+Turkey+Pouch+Stage+3'}],
 ['1025204', {'Graduates+Fruit+Pick-Ups+Diced+Apples'}],
 ['1026466', {'Organic+Go+Go+Apple+Peach+Applesauce'}],
 ['102931',
  {'Broccoli+&+Apple+Stage+2+Baby+Food', 'Honest+Face,+Hand,+&+Baby+Wipes'}],
 ['1029353', {'Whole+Grain+Rice+Cereal'}],
 ['1033471', {'Blueberry+&+Purple+Sweet+Potato++Super+Puffs'}],
 ['103546',
  {'Baby+Food+Stage+2+Bluebe

In [6]:
I

['1pk+270ct+Refill',
 '1pk+270ct+Refill',
 '1st+Foods+Applesauce+Baby+Food',
 '1st+Foods+Baby+Food-+Pears',
 '1st+Foods+Baby+Food-+Pears',
 '1st+Foods+Bananas+Baby+Food',
 '1st+Foods+Bananas+Baby+Food',
 '1st+Foods+Bananas+Baby+Food',
 '1st+Foods+Green+Beans',
 '1st+Foods+Green+Beans',
 '1st+Foods+Peaches',
 '1st+Foods+Squash',
 '1st+Foods+Sweet+Potatoes',
 '1st+Foods+Sweet+Potatoes',
 '2nd+Food+Organic+Pear+Peach+&+Strawberries',
 '2nd+Foods+Apple+Strawberry+Banana',
 '2nd+Foods+Apples',
 '2nd+Foods+Apples+&+Mangos,+With+Rice+Cereal',
 '2nd+Foods+Banana+Mixed+Berries',
 '2nd+Foods+Chicken+&+Gravy',
 '2nd+Foods+Chicken+&+Gravy',
 '2nd+Foods+Chicken+&+Rice+Baby+Food',
 '2nd+Foods+Green+Beans+Baby+Food',
 '2nd+Foods+Green+Beans+Baby+Food',
 '2nd+Foods+Nature+Select+Blended+Fruits+with+Oatmeal',
 '2nd+Foods+Organic++Pear+and+Spinach+Baby+Food',
 '2nd+Foods+Organic+Apples,+Carrots+&+Squash+Baby+Food',
 '2nd+Foods+Organic+Fruit+&+Veggies+-+Apples,+Zucchini,+Spinach+&+Strawberries',
 '2nd+Fo

5. Напишем метод Apriori и проверим его на тестовых данных 

In [7]:
from itertools import chain, combinations
from anytree import Node

def ComputeSupport( X, D ):
    supX = 0
    for _,itemset in D:
        if X.issubset( itemset ):
            supX += 1
    return supX

def ExtendPrefixTree(Ck):

    tmp_ck = []
    for i in range(len(Ck)):
        for j in range(i + 1, len(Ck)):
            flag = True
            val = sorted(set(Ck[i].name) | set(Ck[j].name))
            if len(val) != (len(Ck[i].name) + 1):
                continue
            for z in tmp_ck:
                if z.name == val:
                    flag = False
                    break
            if flag:
                tmp_ck.append(Node(val, Ck[i]))
    return tmp_ck


def Apriori(D, I, minsup):
    F = []
    Ck = []
    root = Node([])
    I = sorted(I)
    for i in I:
        Ck.append(Node([i], parent=root))
    k = 1
    while Ck != []:
        i = 0
        while i < len(Ck):
            X = Ck[i]
            supX = ComputeSupport(set(X.name), D)
            if supX >= minsup:
                added_data = [tuple(X.name), supX]
                if added_data not in F:
                    F.append(added_data)
                i += 1
            else:
                X.parent = None
                Ck.remove(X)
        Ck = ExtendPrefixTree(Ck)
    return F

In [8]:
D_train = [
    [ 1, {"A","B","D","E"} ],
    [ 2, {"B","C","E"} ],
    [ 3, {"A","B","D","E"} ],
    [ 4, {"A","B","C","E"} ],
    [ 5, {"A","B","C","D","E"} ],
    [ 6, {"B","C","D"} ],
]
I_train = {"A","B","C", "D", "E"}
Apriori(D_train, I_train, 3)

[[('A',), 4],
 [('B',), 6],
 [('C',), 4],
 [('D',), 4],
 [('E',), 5],
 [('A', 'B'), 4],
 [('A', 'D'), 3],
 [('A', 'E'), 4],
 [('B', 'C'), 4],
 [('B', 'D'), 4],
 [('B', 'E'), 5],
 [('C', 'E'), 3],
 [('D', 'E'), 3],
 [('A', 'B', 'D'), 3],
 [('A', 'B', 'E'), 4],
 [('A', 'D', 'E'), 3],
 [('B', 'C', 'E'), 3],
 [('B', 'D', 'E'), 3],
 [('A', 'B', 'D', 'E'), 3]]

6. При помощи метода Apriori построим популярный набор предметов с минимальной поддержкой не менее 3, имеющий максимальную длину.

In [9]:
minsup = 3
items = Apriori(tdbase, I, minsup )

In [10]:
print("\nПопулярные наборы предметов для minsup =", minsup, ":") 
for itemset in items:
    print(itemset)


Популярные наборы предметов для minsup = 3 :
[('1st+Foods+Bananas+Baby+Food',), 3]
[('AdvanceCare+Oral+Electrolyte+Solution+-+Cherry+Punch',), 6]
[('Apple+Blueberry+Fruit+Yogurt+Smoothie',), 5]
[('Apple+Peach+Oatmeal+Stage+2',), 11]
[('Apple+Puffs+Finger+Foods',), 6]
[('Apple+and+Carrot+Stage+2+Baby+Food',), 10]
[('Apples+&+Blueberries+Stage+2+Baby+Food',), 9]
[('Apples+++Strawberries+Organic+Nibbly+Fingers',), 18]
[('Apples,+Guavas+&+Beets+Organic+Baby+Food',), 8]
[('Apples,+Kale+&+Avocados+Organic+Baby+Food',), 14]
[('Apples,+Pumpkin+&+Carrots+Organic+Baby+Food',), 13]
[('Baby+Dry+Diapers+Jumbo+Pack+Size+4',), 3]
[('Baby+Food+Meals',), 3]
[('Baby+Food+Pears+Squash',), 10]
[('Baby+Food+Pouch+-+Butternut+Squash,+Carrot+&+Chickpea',), 4]
[('Baby+Food+Pouch+-+Kale,+Sweet+Corn+&+Quinoa',), 4]
[('Baby+Food+Pouch+-+Roasted+Carrot+Spinach+&+Beans',), 4]
[('Baby+Food+Stage+2+Blueberry+Pear+&+Purple+Carrot',), 45]
[('Baby+Food+Stage+2+Pumpkin+Banana',), 22]
[('Baby+Food+Stage+2+Raspberry+Spin

7. Построим набор ассоциативных правил

In [11]:
def powersetk(iterable,k):
    xs = list(iterable)
    # возвращаем итератор, а не список
    return chain.from_iterable(combinations(xs,n) for n in range(k,len(xs)+1))

def AssociationRules(D, Z_set, minconf):
    A_rules = []
    supZ = ComputeSupport(set(Z_set), D)
    A_set = list(powersetk(Z_set,1))[:-1]
    # print("\nA_set:",A_set)
    while len(A_set)>0:
        X_set = A_set[-1]
        #print("\nX_set:",X_set)
        A_set.pop()
        conf = supZ/ComputeSupport(set(X_set), D)
        if conf >= minconf:
            Y_set = sorted(list(set(Z_set)-set(X_set)))
            A_rules.append([X_set, Y_set, supZ, conf])
        else:
            for W_set in powersetk(X_set,1):
                if W_set in A_set:
                    A_set.remove(W_set)
    return A_rules

F_set, _ = items[-1]
AR = AssociationRules(tdbase, F_set, 0.7)
print("Ассоциативные правила:")
for i in AR:
    print(i)

Ассоциативные правила:
[('Apples,+Kale+&+Avocados+Organic+Baby+Food', 'Apples,+Pumpkin+&+Carrots+Organic+Baby+Food', 'Baby+Food+Pears+Squash', 'Stage+2+Pear+Pumpkin+Baby+Food'), ['Apples,+Guavas+&+Beets+Organic+Baby+Food'], 3, 1.0]
[('Apples,+Guavas+&+Beets+Organic+Baby+Food', 'Apples,+Pumpkin+&+Carrots+Organic+Baby+Food', 'Baby+Food+Pears+Squash', 'Stage+2+Pear+Pumpkin+Baby+Food'), ['Apples,+Kale+&+Avocados+Organic+Baby+Food'], 3, 1.0]
[('Apples,+Guavas+&+Beets+Organic+Baby+Food', 'Apples,+Kale+&+Avocados+Organic+Baby+Food', 'Baby+Food+Pears+Squash', 'Stage+2+Pear+Pumpkin+Baby+Food'), ['Apples,+Pumpkin+&+Carrots+Organic+Baby+Food'], 3, 1.0]
[('Apples,+Guavas+&+Beets+Organic+Baby+Food', 'Apples,+Kale+&+Avocados+Organic+Baby+Food', 'Apples,+Pumpkin+&+Carrots+Organic+Baby+Food', 'Stage+2+Pear+Pumpkin+Baby+Food'), ['Baby+Food+Pears+Squash'], 3, 1.0]
[('Apples,+Guavas+&+Beets+Organic+Baby+Food', 'Apples,+Kale+&+Avocados+Organic+Baby+Food', 'Apples,+Pumpkin+&+Carrots+Organic+Baby+Food', 'Ba

8. Для построенного набора ассоциативных правил вычислим показатели (меры) оценки ассоциативных правил, указанные в индивидуальном задании (достоверность, убежденность).

In [12]:
def rsup(X, D):
    return ComputeSupport(X, D) / len(D)
def conf(X, Y, D):
    s = ComputeSupport(X | Y, D)
    c = s/ComputeSupport(X, D)
    return c
    
def conv(X,Y, D):
    val1 = 1 - rsup(Y,D)
    val2 = 1 - conf(X,Y,D)
    if val2 == 0:
        return "infinity"
    return val1/val2

In [13]:
for i in range(len(AR)):
    conf_val = conf(set(AR[i][0]), set(AR[i][1]), tdbase)
    conv_val = conv(set(AR[i][0]), set(AR[i][1]), tdbase)
    print("Достоверность: {}. Убежденность: {}.".format(conf_val, conv_val))

Достоверность: 1.0. Убежденность: infinity.
Достоверность: 1.0. Убежденность: infinity.
Достоверность: 1.0. Убежденность: infinity.
Достоверность: 1.0. Убежденность: infinity.
Достоверность: 1.0. Убежденность: infinity.
Достоверность: 1.0. Убежденность: infinity.
Достоверность: 1.0. Убежденность: infinity.
Достоверность: 1.0. Убежденность: infinity.
Достоверность: 1.0. Убежденность: infinity.
Достоверность: 1.0. Убежденность: infinity.
Достоверность: 1.0. Убежденность: infinity.
Достоверность: 1.0. Убежденность: infinity.
Достоверность: 1.0. Убежденность: infinity.
Достоверность: 1.0. Убежденность: infinity.
Достоверность: 1.0. Убежденность: infinity.
Достоверность: 0.75. Убежденность: 3.98728813559322.
Достоверность: 1.0. Убежденность: infinity.
Достоверность: 0.75. Убежденность: 3.98728813559322.
Достоверность: 0.75. Убежденность: 3.98728813559322.
Достоверность: 0.75. Убежденность: 3.98728813559322.
